# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>



## Wikipedia page views (SMAPE metric)

In [1]:
import pandas as pd
import numpy as np

In [425]:
### This is data from wikipedia3 archive in data folder
np.set_printoptions(suppress=True)
train = pd.read_csv("../data/wikipedia_train3.csv",parse_dates=['date'])
test = pd.read_csv("../data/wikipedia_test3.csv",parse_dates=['date'])

In [3]:
train.iloc[0]

Page      Special:Search_zh.wikipedia.org_all-access_spider
date                                    2016-01-01 00:00:00
Visits                                                 2660
Name: 0, dtype: object

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

In [4]:
train_unmelted = train.set_index(['Page', 'date'])['Visits'].unstack().reset_index()


In [5]:
test_unmelted = test.set_index(['Page', 'date'])['Visits'].unstack().reset_index()

In [6]:
interval_train = pd.to_datetime(train.iloc[-1].date) - pd.to_datetime(train.iloc[0].date)
interval_test = pd.to_datetime(test.iloc[-1].date) - pd.to_datetime(test.iloc[0].date)
print("Range of days [train {}] [test {}] we are working with".format(interval_train.days,interval_test.days))

Range of days [train 243] [test 61] we are working with


In [7]:
date_end = train.iloc[-1].date
date_start = test.iloc[0].date
date_diff = (date_start-date_end).days
print("Last day on training set {}".format(date_end))
print("First day in test set {}".format(date_start))
print("Days difference -  {} ".format(date_diff))

Last day on training set 2016-08-31 00:00:00
First day in test set 2016-09-10 00:00:00
Days difference -  10 


In [9]:
### Your code here
from dateutil.relativedelta import relativedelta

def create_validation(df, start_date,day_interval=1,valid_size_coef=1,train_test_diff=10):
    return df.loc[(df['date'] >= pd.to_datetime(start_date) - relativedelta(days=0)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(days=day_interval))].index, \
           df.loc[(df['date'] >= pd.to_datetime(start_date) + relativedelta(days=day_interval+train_test_diff-1)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(days=day_interval+train_test_diff-1+int(day_interval*valid_size_coef)))].index

In [10]:
dates = pd.date_range(start=train.iloc[0].date,end=train.iloc[-1].date,freq="1D")

In [462]:
myCViterator = []
for date in dates:
    ##Ten days slide is preserved , also the ratio 1/4 is kept (due to the train/test ratio)
    trainIndices, valIndices = create_validation(train,date,day_interval=30,valid_size_coef=1/4)
    myCViterator.append( (trainIndices, valIndices))
myCViterator = [val for val in myCViterator if len(val[1]>0)]    

In [457]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

In [458]:
validation_scores = []
for pair in myCViterator:
    result = train.iloc[pair[0]].groupby("Page").median().reset_index()
    merged = pd.merge(train.iloc[pair[1]],result,on=["Page"])
    merged.rename(index=str, columns={"Visits_x": "Visits", "Visits_y": "pred_Visits"},inplace=True)
    validation_scores.append(pandas_smape(merged))


In [459]:
valid_score = np.mean(validation_scores)

In [460]:
print("Validation  SMAPE  on rolling medians {}".format(valid_score))

Validation  SMAPE  on rolling medians 34.80782207013901


#### Task 3 - 15 days median prediction

In [28]:

data = train_unmelted[[train_unmelted.columns[0]] +list(train_unmelted.columns[-15:])]
data = data.melt(id_vars=["Page"]).groupby("Page").median().reset_index()
result = pd.merge(test,data,on=["Page"])
result.rename(index=str, columns={"Visits_x": "Visits", "value": "pred_Visits"},inplace=True)

In [29]:
print("SMAPE of 15 days prediction median {} ".format(pandas_smape(result)))

SMAPE of 15 days prediction median 40.00701270204255 


In [461]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

**3.** Perform K-fold validation using your type of split. Run GridSearch with any regressor you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

In [431]:
from sklearn.tree  import DecisionTreeRegressor
regressor = DecisionTreeRegressor()

In [468]:
import math
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [469]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
my_scorer = make_scorer(smape_fast,greater_is_better=False)

In [434]:
def encode_dataframe(df,cols):
    for col in cols:
        df[col]=  df[col].astype('category')
    cat_columns = df.select_dtypes(['category']).columns
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

In [426]:
def prepare_for_model(df):
    df2= df.copy()
    df2['Month']     = df2["date"].dt.month
    df2['Day']       = df2["date"].dt.day
    df2['DayOfWeek'] = df2["date"].dt.dayofweek
    return df2

def split_X_Y(df):
    encode_dataframe(df,["Page"])
    X = df.drop(["date","Visits"],axis=1)
    Y=df["Visits"]
    return (X,Y)

In [466]:
train_model,test_model = prepare_for_model(train),prepare_for_model(test)
X_train,Y_train= split_X_Y(train_model)
X_test,Y_test = split_X_Y(test_model)

In [470]:
scores = cross_val_score(regressor,X_train,Y_train,scoring=my_scorer,cv=myCViterator,n_jobs=8)

In [471]:
valid_score= np.abs(np.mean(scores))
print("Validation score on model {}".format(valid_score))

Validation score on model 41.17666386486947


In [449]:
regressor.fit(X_train,Y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [450]:
predictions=regressor.predict(X_test)


In [451]:
test_score  = smape_fast(Y_test,predictions)

In [452]:
print("Test SMAPE score {}".format(test_score))

Test SMAPE score 44.92904614403156


In [472]:
print("The difference here is {:.3f}%".format(np.abs(test_score-valid_score)))


The difference here is 3.752%


## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>



In [2]:
train_house = pd.read_csv('../data/train_french.csv')
test_house = pd.read_csv('../data/test_french.csv')

In [23]:
import pandas_profiling as pp
pp.ProfileReport(train_house)


Number of variables,29
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,44.3 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


We see a lot of correlated features. 

In [27]:
encode_cols=["veranda","sit_fam","nature_lri","niv_urbanisation","var9","profession"]



In [4]:
for dframe in [train_house,test_house]:
    encode_dataframe(dframe,encode_cols)

In [5]:
train_house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 29 columns):
code_postal          200000 non-null int64
nb_pieces            200000 non-null int64
veranda              200000 non-null int8
biens_nomades        200000 non-null int64
sit_fam              200000 non-null int8
profession           200000 non-null int8
nature_lri           200000 non-null int8
societe              200000 non-null int64
domaine_activite     200000 non-null int64
niv_urbanisation     200000 non-null int8
age_societaire       200000 non-null int64
objets_precieux      200000 non-null int64
pres_contrat_auto    200000 non-null int64
formule              200000 non-null int64
sit_juridique        200000 non-null int64
surf_habitat         200000 non-null int64
patrimoine_mob       200000 non-null int64
var1                 200000 non-null int64
var2                 200000 non-null int64
var3                 200000 non-null int64
var4                 200000 non-nu

All categorical features were converted to numerical ones, we're good to go

In [97]:
split_index = len(train_house)

In [103]:
general_table = pd.concat([train_house,test_house])

In [108]:
condition= all(general_table[:len(train_house)]==train_house) &\
           all(general_table[len(train_house):]==test_house)
    
print("DFs are concataned properly - {}".format(condition))    

DFs are concataned properly - True


In [149]:
def first_n_digits(num, n):
    return num // 10 ** ((np.log10(num) - n + 1).astype(np.int32))
def get_last_digits(num, n):
    return abs(num) % (10**n)

In [157]:
general_table["pc_region"] = first_n_digits(general_table.code_postal.values,2)
general_table["pc_subregions"] = get_last_digits(general_table.code_postal.values,3)
general_table["rooms_per_area"] = general_table["nb_pieces"]/general_table["surf_habitat"]


In [163]:
train_house = general_table[:split_index]
test_house = general_table[split_index:]

In [148]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score

In [149]:
regressor  = RandomForestRegressor()
regressor_simple = DecisionTreeRegressor()

In [38]:
def mean_absolute_percentage_error(y_true, y_pred): 
    #y_true, y_pred = check_arrays(y_true, y_pred)
    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [154]:
from sklearn.metrics import  make_scorer

In [44]:
my_scorer = make_scorer(mean_absolute_percentage_error,greater_is_better=False)

In [151]:
param_grid = {
              "min_samples_split": list(range(2,10)),
              "min_samples_leaf":list(range(2,12,2)) ,
             }

I've experimented here with multiple params, and found that limiting  *max_features*  or depth of the tree only makes the performance worse. Thus, I mainly runing *min_samples_split , min_samples_leaf*  parameters.

In [175]:
grid_search = GridSearchCV(regressor_simple, param_grid=param_grid,scoring= my_scorer,verbose=3,cv=3,n_jobs=8)

In [176]:
Y_train,Y_test = train_house.prime_tot_ttc,test_house.prime_tot_ttc
X_train ,X_test= train_house.drop(["prime_tot_ttc"],axis=1),test_house.drop(["prime_tot_ttc"],axis=1)

In [177]:
grid_search.fit(X_train,Y_train)

Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] min_samples_split=2, min_samples_leaf=2 .........................
[CV] min_samples_split=2, min_samples_leaf=2 .........................
[CV] min_samples_split=2, min_samples_leaf=2 .........................
[CV] min_samples_split=3, min_samples_leaf=2 .........................
[CV] min_samples_split=3, min_samples_leaf=2 .........................
[CV] min_samples_split=3, min_samples_leaf=2 .........................
[CV] min_samples_split=4, min_samples_leaf=2 .........................
[CV] min_samples_split=4, min_samples_leaf=2 .........................
[CV]  min_samples_split=2, min_samples_leaf=2, score=-11.13817366832283, total=  11.8s
[CV] min_samples_split=4, min_samples_leaf=2 .........................
[CV]  min_samples_split=2, min_samples_leaf=2, score=-11.250178601880247, total=  12.4s
[CV] min_samples_split=5, min_samples_leaf=2 .........................
[CV]  min_samples_split=3, min_samples_leaf=2, score=

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   28.9s


[CV] min_samples_split=9, min_samples_leaf=2 .........................
[CV]  min_samples_split=7, min_samples_leaf=2, score=-11.04710490521104, total=  12.9s
[CV] min_samples_split=2, min_samples_leaf=4 .........................
[CV]  min_samples_split=7, min_samples_leaf=2, score=-10.912993809476337, total=  13.0s
[CV]  min_samples_split=8, min_samples_leaf=2, score=-10.874812988034014, total=  12.6s
[CV] min_samples_split=2, min_samples_leaf=4 .........................
[CV] min_samples_split=2, min_samples_leaf=4 .........................
[CV]  min_samples_split=8, min_samples_leaf=2, score=-10.980782922919254, total=  12.9s
[CV]  min_samples_split=8, min_samples_leaf=2, score=-10.842230339993755, total=  12.8s
[CV]  min_samples_split=9, min_samples_leaf=2, score=-10.805495509430218, total=  12.7s
[CV] min_samples_split=3, min_samples_leaf=4 .........................
[CV] min_samples_split=3, min_samples_leaf=4 .........................
[CV]  min_samples_split=9, min_samples_leaf=2, 

[Parallel(n_jobs=8)]: Done 120 out of 120 | elapsed:  3.1min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9], 'min_samples_leaf': [2, 4, 6, 8, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False),
       verbose=3)

In [181]:
np.abs(grid_search.best_score_)

10.748583437499263

In [182]:
params = grid_search.best_params_

In [183]:
params

{'min_samples_leaf': 10, 'min_samples_split': 8}

In [184]:
gs_tree = DecisionTreeRegressor(**params)

In [185]:
scores = cross_val_score(gs_tree, X_train, Y_train, cv=3,scoring=my_scorer)

In [186]:
print("MAPE validation score {}".format(np.abs(np.mean(scores))))

MAPE validation score 10.748989511917912


In [209]:
gs_tree.fit(X_train,Y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=10,
           min_samples_split=8, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [188]:
predictions = gs_tree.predict(X_test)

In [189]:
print("MAPE test score {}".format(mean_absolute_percentage_error(Y_test,predictions)))

MAPE test score 10.006008557585789


In [222]:
np.set_printoptions(suppress=True)
features = list(zip(X_train.columns,gs_tree.feature_importances_))

In [223]:
from operator import itemgetter

In [224]:
list(reversed(sorted(features,key = itemgetter(1))))

[('biens_nomades', 0.31163949209144837),
 ('formule', 0.24641268102998234),
 ('code_postal', 0.16270747198275884),
 ('patrimoine_mob', 0.08608273340956095),
 ('pc_subregions', 0.06684649930879555),
 ('sit_juridique', 0.04857607238541409),
 ('pc_region', 0.03984711542315239),
 ('sit_fam', 0.005675122555294623),
 ('surf_habitat', 0.004506783894150927),
 ('rooms_per_area', 0.004310876863190901),
 ('var4', 0.004257596579503386),
 ('age_societaire', 0.0037189864200969573),
 ('niv_urbanisation', 0.002366593241898189),
 ('domaine_activite', 0.0023258828477856334),
 ('profession', 0.0015660060321470962),
 ('nb_pieces', 0.0015266573821061905),
 ('var10', 0.0013273227664631444),
 ('var5', 0.0010674062812302254),
 ('var1', 0.0009322085792354413),
 ('var11', 0.0009019108774211765),
 ('var6', 0.0008056907541464221),
 ('var2', 0.0007499708879324655),
 ('nature_lri', 0.0007363891450353596),
 ('var9', 0.0003135122623633341),
 ('societe', 0.0002655409566131729),
 ('pres_contrat_auto', 0.000141914295525